In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [4]:
games_df = pd.read_csv("Resources/startercsv/all_video_games(cleaned).csv", converters={'Platforms Info': literal_eval})
games_df

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/startercsv/all_video_games(cleaned).csv'

In [5]:
games_df.info()

NameError: name 'games_df' is not defined

In [ ]:
games_df['Release Date'] = pd.to_datetime(games_df['Release Date'])
games_df

In [ ]:
games_df.info()

In [ ]:
games_df.count()

In [ ]:
games_df['Product Rating'].unique()

In [ ]:
print(games_df[games_df['Title'].isnull()])

In [ ]:
games_df = games_df.dropna(subset=["Title"])
games_df = games_df.reset_index(drop=True)

In [ ]:
games_df.count()

In [ ]:
# Assigning 'Platforms Info to a variable'
pinfo = games_df['Platforms Info']

# Empty lists to store extracted data from 'Platform Info' column
meanmeta = []
platlist = []
metascore = []
metacount = []

# Looping through list of games
for game in pinfo:
    
    # Extracting platform names to a list
    platform = ([x.get('Platform') for x in game])
    platlist.append(platform)
    
    # Extracting scores to a list and converting to number
    # Calculting average metascore across platforms
    score = ([y.get('Platform Metascore') for y in game])
    scorelist = []
    avglist = []
    meanvalue = []
    for item in score:    
        num = pd.to_numeric(item, errors='ignore')
        # Appending to list that holds all values of metascore
        scorelist.append(num)
        
        #Appending to list that holds only numeric values of metascores
        if num == "tbd":
            avglist = avglist
        else:
            avglist.append(num)
            
    # Calculating the average metascore per game        
    if avglist == []:
        meanvalue = 0
    else:
        meanvalue = sum(avglist)/len(avglist)
        meanvalue = round(meanvalue,1)
    meanmeta.append(meanvalue)
    metascore.append(scorelist)
    
    # Extracting review counts from string, converting to number, adding to a list
    counts = ([z.get('Platform Metascore Count') for z in game])
    numlist = []
    for item in counts:
        num = [int(i) for i in item.split() if i.isdigit()]
        newnum = num[0]
        numlist.append(newnum)
    metacount.append(numlist)
    
# Adding new columns to df
games_df["Mean Metascore Across Platforms"] = meanmeta
games_df['Platform'] = platlist
games_df['Metascore by Platform'] = metascore
games_df['Count of Critic Reviews by Platform'] = metacount

games_df


In [ ]:
games_df.columns.values.tolist()


In [ ]:
# [nan, 'Rated E For Everyone', 'Rated T For Teen',
#        'Rated E +10 For Everyone +10', 'Rated M For Mature',
#        'Rated RP For Rate Pending', 'Rated AO For Adults Only'
# Simplifying Product Rating
games_df['Product Rating Simplified'] = ''


games_df.loc[games_df['Product Rating'] == 'Rated E For Everyone', ['Product Rating Simplified']] = 'E'
games_df.loc[games_df['Product Rating'] == 'Rated T For Teen', ['Product Rating Simplified']] = 'T'
games_df.loc[games_df['Product Rating'] == 'Rated E +10 For Everyone +10', ['Product Rating Simplified']] = 'E+10'
games_df.loc[games_df['Product Rating'] == 'Rated M For Mature', ['Product Rating Simplified']] = 'M'
games_df.loc[games_df['Product Rating'] == 'Rated RP For Rate Pending', ['Product Rating Simplified']] = 'RP'
games_df.loc[games_df['Product Rating'] == 'Rated AO For Adults Only', ['Product Rating Simplified']] = 'AO'

games_df = games_df[['Title','Release Date','Developer','Publisher','Genres','Product Rating',
             'Product Rating Simplified','User Score','User Ratings Count','Platforms Info',
             'Mean Metascore Across Platforms','Platform','Metascore by Platform','Count of Critic Reviews by Platform']]

In [ ]:
games_df

In [ ]:
# Finding Unique Platforms

games = games_df['Platform']

biglist = []
for game in games:
    for item in game:
        biglist.append(item)
        

import operator as op
def unique(list1):
    # initialize list
    unique_list = []
    
    # iterate over list
    for x in list1:
        # check if exists in unique_list or not
        if op.countOf(unique_list, x) == 0:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)

unique(biglist)

In [ ]:
# unique Game Genres

genres = games_df['Genres'].unique()
genres

In [ ]:
games_df.info()

In [ ]:
games_df.to_csv("Resources/games.csv", index=False, header=True)

In [ ]:
# release date column to datetime
games_df['Release Date']= pd.to_datetime(games_df['Release Date'])

# get year from Release Date column
games_df['Year']=games_df['Release Date'].dt.year

# Drop rows with NaN values in User Score column
games_df = games_df.dropna(subset=['User Score'])

# Filter the dataset for time period (2023 to 2024)
games_filtered = games_df[(games_df['Release Date'].dt.year >= 2023) & (games_df['Release Date'].dt.year <= 2024)]

# Group by year and publisher and calculate average user score
avg_scores_publisher_year = games_filtered.groupby(['Year', 'Publisher'])['User Score'].mean().reset_index()

# Group by publisher and calculate average user score across all years
avg_scores_publisher = avg_scores_publisher_year.groupby('Publisher')['User Score'].mean().reset_index()

# Avg scroes by publisher
avg_scores_publisher = avg_scores_publisher.sort_values(by='User Score', ascending=False)

# Create bar graph
plt.figure (figsize=(10,20)) 

bars = plt.barh(avg_scores_publisher['Publisher'],avg_scores_publisher['User Score'], color='lightblue')

# Ttitle and labels
plt.title('Average User Score Rating by Publisher (2023 - 2024)', fontsize=20)
plt.xlabel('Average User Score', fontsize=16)
plt.ylabel('Publisher', fontsize=16)

# Add Values
for bar in bars:
    width = bar.get_width()
    plt.text(width,bar.get_y()+ bar.get_height()/2, round(width, 2), ha='left', va='center', fontsize=7)

# Display Chart and save figure
plt.grid(axis='x') 
plt.tight_layout() 
plt.show()
plt.savefig('avg_user_score.png', dpi=300) 

In [ ]:
games_df = games_df.copy()
games_df['Release Date'] = pd.to_datetime(games_df['Release Date'])

games_df['Year'] = games_df['Release Date'].dt.year

# Group by year and publisher and count the number of releases
release_yr_publisher = games_df.groupby(['Year', 'Publisher']).size().reset_index(name='Release Count')

# Create trace for each publisher
data = []
for publisher in release_yr_publisher['Publisher'].unique():
    publisher_data = release_yr_publisher[release_yr_publisher['Publisher'] == publisher]
    trace = go.Bar(x=publisher_data['Year'],
                   y=publisher_data['Release Count'],
                   name=publisher)
    data.append(trace)

# Create layout
layout = go.Layout(title='Game Releases by Publisher over Time',
                   xaxis=dict(title='Year of Release'),
                   yaxis=dict(title='Number of Releases'),
                   barmode='stack')
fig = go.Figure(data=data, layout=layout)
fig.show()
plt.savefig('release_publisher_ot.png') 

In [ ]:
genre_count_yr = games_df.groupby(['Year', 'Genres']).size().reset_index(name='Genre Count')

# Sum the counts across all years for each genre
genre_total_count = genre_count_yr.groupby('Genres')['Genre Count'].sum().reset_index()

# Sort the genres based on their total counts in descending order
top_10 = genre_total_count.nlargest(5, 'Genre Count')

# Filter the original dataframe to include only the top 10 genres
top_10_genre_by_year = genre_count_yr[genre_count_yr['Genres'].isin(top_10['Genres'])]

# Create an interactive bar graph using Plotly
fig = px.bar(top_10_genre_by_year, x='Year', y='Genre Count', color='Genres',
             title='Count of Top 5 Genres Over Time',
             labels={'Year': 'Year', 'Genre Count': 'Count of Genres', 'Genres': 'Genre'},
             barmode='group') 

# Show the chart
fig.show()

In [ ]:
filtered_data = games_df[(games_df['Year'] >= 1995) & (games_df['Year'] <= 2024)]

# Group year and product rating
product_rating_yr = filtered_data.groupby (['Year','Product Rating']).size().reset_index(name ='Rating Count')

# Create graph - Line
fig = go.Figure()

for rating in product_rating_yr['Product Rating'].unique():
    rating_data = product_rating_yr[product_rating_yr['Product Rating'] == rating]
    fig.add_trace(go.Scatter(x=rating_data['Year'], y=rating_data['Rating Count'], mode='lines',name=rating))
    
# Layout
fig.update_layout(title='product Rating Count by Year 1995 - 2024',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Rating Count'))

fig.show(renderer="png")